In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import cloudvolume as cv
import torch

from visualize import PyramidVisualizer, get_dataset, simple_visualizer
from metroem.aligner import Aligner

/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [49]:
#raw_img_path = 'gs://corgie_package/paper/basil_data_x0/aligned_x0/img/img_full_x0_rend'
raw_img_path = 'https://s3-hpcrc.rc.princeton.edu/fafbv15-em/v1'
raw_img_path = 'gs://tmacrina-corgie-test/fafbv15/precoarse_v15-1/sample/onepass_test/img/img_v2'

#raw_img_path = 'gs://tmacrina-corgie-test/fafbv15/precoarse_v15-1/sample/img/img_normalized'
#masked_img_path = 'gs://corgie_package/paper/basil_data_x0/img/img_normed_with_cracks'
#field_path = 'gs://corgie_package/paper/basil_data_x0/aligned_x0/field/field_full_x0'
#masked_img_mip = 4
img_mip = 4
#field_mip = 4
z = 165
x_start = 140000
y_start = 50000
x_size = 1024 * 2**img_mip
y_size = x_size
pad = 64 * 2**3
#
#img_cv = cv.CloudVolume(raw_img_path, mip=img_mip, parallel=True, progress=False)
#masked_img_cv = cv.CloudVolume(masked_img_path, mip=masked_img_mip, parallel=True, progress=False)
#field_cv = cv.CloudVolume(field_path, mip=field_mip, parallel=True, progress=False)

In [50]:
img_cv = cv.CloudVolume(raw_img_path, mip=img_mip, parallel=True, progress=False)

In [51]:
import metroem


In [52]:
img1_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z:z+1].squeeze()
img2_np = img_cv[(x_start-pad)//2**img_mip:(x_start+x_size+pad)//2**img_mip, (y_start-pad)//2**img_mip:(y_start+y_size+pad)//2**img_mip, z+1:z+2].squeeze()

In [53]:
img1.shape

torch.Size([1, 1, 1088, 1088])

In [54]:
img1 = torch.tensor(img1_np).float().cuda().unsqueeze(0).unsqueeze(0)
img2 =  torch.tensor(img2_np).float().cuda().unsqueeze(0).unsqueeze(0)
img1 = metroem.helpers.normalize(img1, mask=img1!=0)
img2 = metroem.helpers.normalize(img2, mask=img2!=0)

In [55]:
simple_visualizer().visualize([
    img1,
    img2
    
])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1), value=0), IntText(value=1, descripti…

In [56]:
model_path = 'gs://tmacrina-corgie-test/models/fly_m4_210601_plain100_metric167/2_mip4in_mip5module'

In [14]:
import modelhouse
import json

stage = 3
checkpoint_name = "test_no_batchnorm"
embedder = modelhouse.loading.uncached_load_model_str(
    #'/usr/people/popovych/metro_models/pure_emb_x1_lite/0_mip7in_purity', 
     model_path, 
     params=json.dumps({
        "checkpoint_name": checkpoint_name,
        "finetune": False
})).cuda()

miss_detect =  modelhouse.loading.uncached_load_model_str("~/models/misalign_detect_fafb_m4", params='{}').cuda()

/usr/people/popovych/.modelhouse/tmp_files/tmp167nataw/model/model_spec.json
block_6convs_fms8to32_skip25.json
categorical/categorical_compch15_gridfalse_maxvalue7_outch2_stepnull_trainsttrue.json
identity.json
embedder_m0_3x3_fms3x.json
block_5convs_3x3_fms1to3_skip14.json
average_pool.json
embedder_fms4to3.json
block_3convs_3x3_fms4to32to3.json
average_pool.json
block_3convs_3x3_fms3to32to3.json
identity.json
identity.json
identity.json
upsample_residuals.json
identity.json


/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/artificery/parsers/categorical_regression/parse.py:38: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  requires_grad=False) - self.component_channels//2


/usr/people/popovych/.modelhouse/tmp_files/tmpkvf3nu77/model/model_spec.json
block_6convs_fms8to32_skip25.json
categorical/categorical_compch15_gridfalse_maxvalue7_outch2_stepnull_trainsttrue.json
identity.json
embedder_m0_3x3_fms3x.json
block_5convs_3x3_fms1to3_skip14.json
average_pool.json
embedder_fms4to3.json
block_3convs_3x3_fms4to32to3.json
average_pool.json
block_3convs_3x3_fms3to32to3.json
identity.json
identity.json
identity.json
upsample_residuals.json
identity.json


/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/artificery/parsers/categorical_regression/parse.py:38: UserWarning: torch.range is deprecated in favor of torch.arange and will be removed in 0.5. Note that arange generates values in [start; end), not [start; end].
  requires_grad=False) - self.component_channels//2


In [ ]:
miss_detect =  modelhouse.loading.uncached_load_model_str("~/models/misalign_detect_fafb_m4", params='{}').cuda()

In [68]:
pred_mask = miss_detect(img1, img2)

In [69]:
emb1.max()*0.6

tensor(0.4321, device='cuda:0')

# Misalignment Detector

In [57]:
import cc3d
import numpy as np
import fastremap 


from metroem.helpers import downsample, upsample, get_np
from metroem.masks import dilate, closing


def filter_small(mask, th=100):
    cc_labels = cc3d.connected_components(mask)
    segids, counts = np.unique(cc_labels, return_counts=True)
    segids = [ segid for segid, ct in zip(segids, counts) if ct > th ]
    filtered_mask = fastremap.mask_except(cc_labels, segids, in_place=True) != 0
    filtered_mask_c = metroem.masks.closing(filtered_mask, n=4)
    return filtered_mask_c

emb1 = upsample(1)(downsample(1)(embedder.aligner.get_embeddings(img1, 0).unsqueeze(0)))[0, 0]
emb2 = upsample(1)(downsample(1)(embedder.aligner.get_embeddings(img2, 0).unsqueeze(0)))[0, 0]

high_r = get_np((emb1 - emb2).abs() > emb1.max()*0.6) 
high_r[get_np(emb1.abs().squeeze() < 0.04)] = 0
high_r[get_np(emb2.abs().squeeze() < 0.04)] = 0

high_r_d = high_r.copy()

for i in range(3):
    high_r_d = filter_small(high_r_d, 4*(12**i))
    high_r_d = dilate(dilate(high_r_d))
    
for _ in range(20):
    high_r_d = dilate(high_r_d)

high_r_d = closing(high_r_d, 8)
no_misd = high_r_d == 0
no_misd_f = filter_small(no_misd, 4000)
missd = no_misd_f == 0


/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
emb1.max()*0.6

In [63]:
simple_visualizer().visualize([
    missd,
    no_misd,
    high_r,
    #high_r_f,
    high_r_d,
    emb1,
    emb2,
    #emb1.abs() < 0.04,
    #(emb1 - emb2).abs() > emb1.max()*0.6,
    (emb1 - emb2).abs(),
    img1,
    img2,
])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3, 4, 5, 6, 7, 8), value=0), IntTe…

In [ ]:
src_emb.max()*0.6

# Resin Detector

In [58]:

resin = get_np(emb1.abs() < 0.05)
resin_f = filter_small(resin, 100)
resin_f = dilate(dilate(resin_f))
resin_f = dilate(dilate(dilate(dilate(dilate(filter_small(resin_f, 4000))))))
tissue = resin_f == 0
tissue = filter_small(tissue, 3000)

/usr/people/popovych/env/corgie_merge/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()


In [61]:
simple_visualizer().visualize([
    tissue,
    resin,
    resin_f,
    img1,
    img11
])

interactive(children=(ToggleButtons(description='Image:', options=(0, 1, 2, 3, 4), value=0), IntText(value=1, …

In [60]:
img11 = img1.clone().squeeze()
img11[tissue == 0] = 0

In [30]:
img1.squeeze().shape

torch.Size([1088, 1088])

In [31]:
tissue.shape

(1088, 1088)

In [34]:
img1.squeeze().shape

torch.Size([1088, 1088])

In [36]:
tissue.shape

(1088, 1088)